In [80]:
import glob
import monai
import os
from os.path import basename, dirname, exists, isdir, join, split
from pathlib import Path
import functools
import multiprocessing as mp
import os
from functools import partial
import numpy as np
import pandas as pd
from toolz.itertoolz import groupby
import SimpleITK as sitk


root_dir='/workspaces/prost_anatomy_seg/data/75'


def group_files(root_dir):
    """ 
    based on name will group the files into triplets of related files
    """
    all_files=os.listdir(root_dir)
    grouped_by_master= groupby(lambda path : path.split('_')[0],all_files)
    return list(dict(grouped_by_master).items())

# https://github.com/jakubMitura14/forPicaiDocker/blob/main/testRadiomics/getGoldLabelsRadiomics.py

def analyze_single_label(uniq_num,centers, big_mask, connected,cover_min,in_min):
    infered=(connected==uniq_num)
    total=np.sum(infered.flatten())
    inn= np.sum(np.logical_and(infered,big_mask).flatten())/total
    cov= np.sum(np.logical_and(infered,centers).flatten())/np.sum(centers.flatten())
    res= (inn>in_min) #and (cov>cover_min)
    return res

def analyze_case(group,root_dir):
    num,paths=group
    paths= list(map(lambda p: f"{root_dir}/{p}",paths))

    centers=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'centers' in pathh  ,paths))[0])).astype(bool)
    big_mask=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'big_mask' in pathh  ,paths))[0])).astype(bool)
    print(np.sum(centers.flatten()))
    if(np.sum(centers.flatten())==0):
        print(f"gggg {num}")
    inferred=sitk.ReadImage(list(filter(lambda pathh: 'inferred' in pathh  ,paths))[0])
    if(np.sum(sitk.GetArrayFromImage(inferred))==0):
        return 1.0
    connected=sitk.GetArrayFromImage(sitk.ConnectedComponent(inferred))
    uniqq=np.unique(connected)
    uniqq= list(filter(lambda el:el>0,uniqq))
    cover_min=0.0
    in_min=0.8
    res= list(map(lambda uniq_num: analyze_single_label(uniq_num,centers, big_mask, connected,cover_min,in_min), uniqq))
    res= np.mean(np.array(res).astype(int))
    return res

grouped=group_files(root_dir)

aa=list(map(lambda g:analyze_case(g,root_dir)  ,grouped))



108
22
63
108
28
21
54
48
84
17
116
26
94
44
31
21
31
21
67
44
31
74
70
201
54
36
94
21
27
201
15
21
59
36
84
116
186
14
11
29
25
39
54
27
54
26
11
67
76
39
94
51
36
48
11
44
112
111
59
84
46
25
14
62
74
54
17
54
74
26
116
186
21
84
84
74
76
25
76
22
44
88
61
36
26
21
84
46
27
76
84
11
26
111
14
46
11
84
59
19
76
84
26
25
74
84
39
108
59
70
62
44
48
109
25
70
70
63
59
76
84
84
74
46
76
67
112
70
63
59
21
26
39
74
54
74
29
186
111
25
19
59
11
44
67
201
61
27
74
74
48
111
39
26
21
60
32
67
54
22
94
44
112
109
19
46
61
76
36
74
76
28
26
25
32
84
54
94
14
11
26
84
48
201
22
31
201
51
51
21
108
26
15
70
21
44
31
74
11
94
186
70
76
88
51
11
17
29
109
19
21
26
31
74
63
25
59
22
31
59
116
67
11
36
27
46
11
28
11
44
18
116
28
186
39
84
26
111
11
31
76
67
84
61
84
108
25
109
62
27
26
201
25
67
60
19
46
44
15
46
60
17
26
63
116
26
88
25
44
67
26
26
26
26
60
94
15
63
46
31
26
61
14
44
61
28
25
21
61
36
54
29
27
109
48
111
84
15
186
63
31
76
61
62
74
48
186
61
48
14
111
84
84
46
109
19
27
14
36
84


In [77]:
np.mean(aa)

0.6831318364997121

In [43]:
g=list(filter(lambda el:el[0]=='5' ,grouped))[0]
analyze_case(g,root_dir)


[1, 2]


('5', 0.5)

In [48]:
np.mean(aa)


0.6862118595279216

In [68]:

def get_is_correct(curr_in,curr_twos):
    total = curr_in.sum()
    curr_percent_in=np.zeros(1)
    curr_percent_covered=np.zeros(1)

    if(total.item()>0):
        curr_percent_in=curr_in.sum()/(total)
    if(curr_twos.sum().item()>0):
        curr_percent_covered=  ((curr_in) & (curr_twos)).sum()/ curr_twos.sum()

    return (curr_percent_in>0.8 and curr_percent_covered>0.4)


def analyze_case_sum(group,root_dir):
    num,paths=group
    paths= list(map(lambda p: f"{root_dir}/{p}",paths))

    centers=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'centers' in pathh  ,paths))[0])).astype(bool)
    big_mask=sitk.GetArrayFromImage(sitk.ReadImage(list(filter(lambda pathh: 'big_mask' in pathh  ,paths))[0])).astype(bool)
    inferred=sitk.ReadImage(list(filter(lambda pathh: 'inferred' in pathh  ,paths))[0])
    connected=sitk.GetArrayFromImage(inferred)
    connected=(connected>0)
    # uniqq= list(filter(lambda el:el>0,uniqq))
    cover_min=0.5
    in_min=0.8
    
    inn = connected & big_mask

    res=get_is_correct(inn,centers)
    # res= analyze_single_label(True,centers, big_mask, connected,cover_min,in_min)
    res= np.mean(np.array(res).astype(int))
    return (num,res)

grouped=group_files(root_dir)

bb=list(map(lambda g:analyze_case_sum(g,root_dir)  ,grouped))

In [69]:
# np.mean(bb)
bbb= list(map(lambda tupl: tupl[1],bb))
# np.mean(bbb)
np.mean(bbb)

0.9499136442141624